# Подключение к базе данных

In [3]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
    "bolt://3.235.199.126:7687", 
    auth=basic_auth("neo4j", "adverb-steel-lieutenant"))
session = driver.session()

# CRUD: create, read, update, delete

Предполагается, что у каждого узла должны быть лейбл и имя, а у каждого отношения - тип.

## Узлы

In [2]:
def create_node(label, name):
    req="CREATE (n:"+label+" {name:'"+name+"'}) RETURN n"
    res = session.run(req)
    for record in res:
        print(record)
    return

def read_node(label, feat_name, feat):
    if label != '':
        label = ':' + label
    
    if feat_name != '':
        req="MATCH (n"+label+") WHERE n."+feat_name+" = '"+feat+"'  RETURN n"
    else:
        req="MATCH (n"+label+") RETURN n"
        
    res = session.run(req)
    for record in res:
        print(record)
    return

def update_node(name, feat_name, feat):
    req="MATCH (n {name:'"+name+"'}) SET n."+feat_name+" = '"+feat+"' RETURN n"
    res = session.run(req)
    for record in res:
        print(record)
    return

def delete_node(label, feat_name, feat):
    if label != '':
        label = ':' + label
    
    if feat_name != '':
        req="MATCH (n"+label+") WHERE n."+feat_name+" = '"+feat+"'  DETACH DELETE n"
    else:
        req="MATCH (n"+label+") DETACH DELETE n"
    return session.run(req)

## Примеры

Добавим еще одну книгу и выведем все узлы с лейблом "книга"

In [4]:
create_node("book", "Tristessa")

<Record n=<Node id=37 labels=frozenset({'book'}) properties={'name': 'Tristessa'}>>


In [5]:
read_node("book", '', '')

<Record n=<Node id=8 labels=frozenset({'book'}) properties={'name': 'On the Road', 'public_year': '1957'}>>
<Record n=<Node id=9 labels=frozenset({'book'}) properties={'name': 'The Dharma Bums', 'public_year': '1958'}>>
<Record n=<Node id=10 labels=frozenset({'book'}) properties={'name': 'Big Sur', 'public_year': '1962'}>>
<Record n=<Node id=11 labels=frozenset({'book'}) properties={'name': 'Desolation Angels', 'public_year': '1965'}>>
<Record n=<Node id=37 labels=frozenset({'book'}) properties={'name': 'Tristessa'}>>


In [ ]:
#delete_node('', 'name', 'Tristessa')

Выведем список альтер-эго одного человека:

In [6]:
read_node('', 'alterego', 'Jack Kerouac')

<Record n=<Node id=15 labels=frozenset({'character'}) properties={'name': 'Sal Paradise', 'alterego': 'Jack Kerouac', 'book': 'On the Road'}>>
<Record n=<Node id=22 labels=frozenset({'character'}) properties={'name': 'Ray Smith', 'alterego': 'Jack Kerouac', 'book': 'The Dharma Bums'}>>
<Record n=<Node id=29 labels=frozenset({'character'}) properties={'book2': 'Desolation Angels', 'name': 'Jack Duluoz', 'alterego': 'Jack Kerouac', 'book': 'Big Sur'}>>


Добавим нового персонажа и укажем, чьим альтер-эго он является:

In [7]:
create_node('character', 'Lazarus Darlovsky')

<Record n=<Node id=40 labels=frozenset({'character'}) properties={'name': 'Lazarus Darlovsky'}>>


In [8]:
update_node('Lazarus Darlovsky', 'alterego', 'Julius Orlovsky')

<Record n=<Node id=40 labels=frozenset({'character'}) properties={'name': 'Lazarus Darlovsky', 'alterego': 'Julius Orlovsky'}>>


In [ ]:
#delete_node_by_name('Lazarus Darlovsky')

Добавим узел нового класса - поэму - и построим отношение, указав, кто ее автор:

In [9]:
create_node('poem', 'Howl')

<Record n=<Node id=41 labels=frozenset({'poem'}) properties={'name': 'Howl'}>>


In [10]:
session.run('MATCH (n:poem), (p:person) WHERE p.name = "Allen Ginsberg" CREATE (p)-[r:IS_AUTHOR_OF]->(n)')

In [ ]:
#delete_node('poem', '', '')

Выведем всех персонажей одной книги:

In [12]:
res = session.run('MATCH (b:book {name:"On the Road"})<-[r:IS_CHARACTER_OF]-(c) RETURN (b)--(c)')
for record in res:
    print(record)

<Record (b)--(c)=[<Path start=<Node id=8 labels=frozenset({'book'}) properties={'name': 'On the Road', 'public_year': '1957'}> end=<Node id=16 labels=frozenset({'character'}) properties={'name': 'Carlo Marx', 'alterego': 'Allen Ginsberg', 'book': 'On the Road'}> size=1>]>
<Record (b)--(c)=[<Path start=<Node id=8 labels=frozenset({'book'}) properties={'name': 'On the Road', 'public_year': '1957'}> end=<Node id=20 labels=frozenset({'character'}) properties={'name': 'Damion', 'alterego': 'Lucien Carr', 'book': 'On the Road'}> size=1>]>
<Record (b)--(c)=[<Path start=<Node id=8 labels=frozenset({'book'}) properties={'name': 'On the Road', 'public_year': '1957'}> end=<Node id=19 labels=frozenset({'character'}) properties={'name': 'Camille', 'alterego': 'Carolyn Cassady', 'book': 'On the Road'}> size=1>]>
<Record (b)--(c)=[<Path start=<Node id=8 labels=frozenset({'book'}) properties={'name': 'On the Road', 'public_year': '1957'}> end=<Node id=18 labels=frozenset({'character'}) properties={'na

## Агрегация и сортировка
Выведем средний возраст

In [13]:
session.run('''MATCH (a:person), (b:person)
WHERE a.death_year <> '' AND b.death_year = ''
SET a.age = abs(toInteger(a.death_year) - toInteger(a.birth_year)), b.age = abs(2021 - toInteger(b.birth_year))
''')
res = session.run('MATCH (p:person) RETURN avg(p.age)')
for record in res:
    print(record)

<Record avg(p.age)=72.19999999999999>


Сортировка по году рождения

In [ ]:
res = session.run('MATCH (p:person) RETURN p.name, p.birth_year ORDER BY p.birth_year, p.name')
for record in res:
    print(record)